In [32]:
import datetime
import tushare as ts
import pymysql

if __name__ == '__main__':

    # 设置tushare pro的token并获取连接
    ts.set_token('**********')
    pro = ts.pro_api()
    # 设定获取日线行情的初始日期和终止日期，其中终止日期设定为昨天。
    start_dt = '20100101'
    time_temp = datetime.datetime.now() - datetime.timedelta(days=1)
    end_dt = time_temp.strftime('%Y%m%d')
    # 建立数据库连接,剔除已入库的部分
    db = pymysql.connect(host='db', user='root', passwd='admin', db='stock', charset='utf8')
    cursor = db.cursor()
    # 设定需要获取数据的股票池
    stock_pool = ['603912.SH','300666.SZ','300618.SZ','002049.SZ','600756.SH','000810.SZ', '603398.SH']
    total = len(stock_pool)
    # 循环获取单个股票的日线行情
    # sql_insert = "CREATE TABLE IF NOT EXISTS stock_all(" \
    #              "state_dt VARCHAR(255) NOT NULL," \
    #              "stock_code VARCHAR(255) NOT NULL," \
    #              "open float," \
    #              "close float," \
    #              "high float," \
    #              "low float," \
    #              "vol float," \
    #              "amount float," \
    #              "pre_close float," \
    #              "amt_change float," \
    #              "pct_change float);"
    # cursor.execute(sql_insert)
    # db.commit()
    
    for i in range(len(stock_pool)):
        try:
            df = pro.daily(ts_code=stock_pool[i], start_date=start_dt, end_date=end_dt)
            # 打印进度
            print('Seq: ' + str(i+1) + ' of ' + str(total) + '   Code: ' + str(stock_pool[i]))
            c_len = df.shape[0]
        except Exception as aa:
            print(aa)
            print('No DATA Code: ' + str(i))
            continue
        for j in range(c_len):
            resu0 = list(df.ix[c_len-1-j])
            resu = []
            for k in range(len(resu0)):
                if str(resu0[k]) == 'nan':
                    resu.append(-1)
                else:
                    resu.append(resu0[k])
            state_dt = (datetime.datetime.strptime(resu[1], "%Y%m%d")).strftime('%Y-%m-%d')
            try:
                sql_insert = "INSERT INTO stock_all(state_dt,stock_code,open,close,high,low,vol,amount,pre_close,amt_change,pct_change) VALUES ('%s', '%s', '%.2f', '%.2f','%.2f','%.2f','%i','%.2f','%.2f','%.2f','%.2f')" % (state_dt,str(resu[0]),float(resu[2]),float(resu[5]),float(resu[3]),float(resu[4]),float(resu[9]),float(resu[10]),float(resu[6]),float(resu[7]),float(resu[8]))
                cursor.execute(sql_insert)
                db.commit()
            except Exception as err:
                continue
    cursor.close()
    db.close()
    print('All Finished!')



Seq: 1 of 7   Code: 603912.SH


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Seq: 2 of 7   Code: 300666.SZ


Seq: 3 of 7   Code: 300618.SZ


Seq: 4 of 7   Code: 002049.SZ


Seq: 5 of 7   Code: 600756.SH


Seq: 6 of 7   Code: 000810.SZ


Seq: 7 of 7   Code: 603398.SH


All Finished!


In [33]:
from sklearn import svm
import DC

if __name__ == '__main__':
    stock = '603398.SH'
    dc = DC.data_collect(stock, '2010-01-01', '2018-09-05')
    train = dc.data_train
    target = dc.data_target
    test_case = [dc.test_case]
    model = svm.SVC()               # 建模
    model.fit(train, target)        # 训练
    ans2 = model.predict(test_case) # 预测
    # 输出涨跌预测，1表示涨，0表示不涨。
    print(ans2[0])



0.0
